In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
from itertools import tee

In [2]:
df = pd.read_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SALUD.csv", delimiter=",")

In [4]:
df_nuevos_salud = pd.read_csv("Nuevos_salud.csv", delimiter=";")

In [53]:
df_nuevos_salud

,id,Name,Latitud,Longitud,concat
0,14,Equipamiento potencial 1,4.119692,-73.620595,4.119692117%2C-73.62059494
1,15,Equipamiento potencial 2,4.124112,-73.617205,4.124111665%2C-73.61720463
2,16,Equipamiento potencial 3,4.128911,-73.608692,4.12891072%2C-73.60869169
3,17,Equipamiento potencial 4,4.126492,-73.619860,4.126492293%2C-73.61986041
4,18,Equipamiento potencial 5,4.107019,-73.655174,4.107019258%2C-73.65517375


In [13]:
array_nuevos_salud=df_nuevos_salud["id"].values

In [14]:
array_nuevos_salud

array([14, 15, 16, 17, 18], dtype=int64)

In [39]:
for col_name in array_nuevos_salud:
    df[col_name] = None

In [1]:
#Se quita por motivos de seguridad
key_API_google="" 

In [50]:
df["concat"]=df['LATITUD'].astype(str)+"%2C"+df['LONGITUD'].astype(str)

In [51]:
origenes= np.array(df["concat"].to_list())

In [52]:
df_nuevos_salud["concat"]=df_nuevos_salud['Latitud'].astype(str)+"%2C"+df_nuevos_salud['Longitud'].astype(str)

In [54]:
destinos=np.array(df_nuevos_salud["concat"].to_list())

In [55]:
todas_solicitudes=[]
for i in range(len(origenes)):
    solicitudes=[]
    for j in range(len(destinos)):
        solicitud="https://maps.googleapis.com/maps/api/distancematrix/json?destinations="+destinos[j]+"&origins="+origenes[i]+"&key="+key_API_google
        solicitudes.append(solicitud)
    todas_solicitudes.append(solicitudes)

In [58]:
todas_solicitudes2 = pd.DataFrame(todas_solicitudes)

In [59]:
def solicitud_maps(url):
    try:
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        x = response.json()
        drive_time_seconds = x['rows'][0]['elements'][0]['duration']['value']
        distance_meters = x['rows'][0]['elements'][0]['distance']['value']
        return [drive_time_seconds, distance_meters]
    except KeyError:
        return 0

In [60]:
df["14"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[0]), axis=1)

In [62]:
df["15"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[1]), axis=1)
df["16"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[2]), axis=1)
df["17"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[3]), axis=1)
df["18"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[4]), axis=1)

In [64]:
df.to_csv("FINAL_tiempos_distancia_saludnuevos.csv",index=True)

In [65]:
df2=df.copy(deep=True)

In [67]:
array_nuevos_salud

array([14, 15, 16, 17, 18], dtype=int64)

In [71]:
df2["15"] = df2["15"].apply(lambda x: x[0])

In [74]:
df2["14"] = df2["14"] .apply(lambda x: x[0])
df2["16"] = df2["16"] .apply(lambda x: x[0])
df2["17"] = df2["17"] .apply(lambda x: x[0])
df2["18"] = df2["18"] .apply(lambda x: x[0])

In [76]:
df2.to_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SALUDNUEVOS.csv",index=True)